In [8]:
# from keras.models import Sequential
# from keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [9]:
# Define the LSTM model using PyTorch
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        predictions = self.fc(lstm_out[:, -1, :])
        return predictions

In [10]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0


In [12]:
# Load the dataset
file_path = 'new.csv'
df = pd.read_csv(file_path)

# Separate features and target
X = df.iloc[:, 1:]  # Exclude the first column which is the target
y = df.iloc[:, 0]   # The first column is the target

# Split the data into training and validation sets
train_size = int(len(df) * 0.8)
X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]
y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]

# Reshape data for LSTM [samples, time steps, features]
X_train_reshaped = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_reshaped = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Further split the training data for time series cross-validation
train_subset_size = int(len(X_train) * 0.5)

# Initialize the PyTorch LSTM model
model_pt = LSTMModel(input_dim=X_train_reshaped.shape[2], hidden_dim=20, output_dim=1)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model_pt.parameters(), lr=0.001)
# Instantiate the EarlyStopping class
early_stopping = EarlyStopping(patience=5, min_delta=0.001)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_reshaped, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)

# Time series cross-validation with the PyTorch LSTM model
results_pt = []

for i in range(train_subset_size, len(X_train) - 80, 80):
    # Define the current training and validation subsets
    X_train_current = X_train_tensor[:i]
    y_train_current = y_train_tensor[:i]
    X_val_current = X_train_tensor[i:i+80]
    y_val_current = y_train_tensor[i:i+80]
    
    # Train the PyTorch LSTM model
    
    for epoch in range(100):  # Reduced number of epochs for faster execution
        model_pt.train()
        optimizer.zero_grad()
        output = model_pt(X_train_current)
        loss = criterion(output, y_train_current)
        loss.backward()
        optimizer.step()
        
        # Evaluate the model on the current validation subset
        model_pt.eval()
        with torch.no_grad():
            y_pred_val_current = model_pt(X_val_current).numpy().flatten()
            val_loss = criterion(torch.tensor(y_pred_val_current), y_val_current).item()
        early_stopping(val_loss)
        if early_stopping.early_stop:
            # print("Early stopping triggered")
            break
    
    # Calculate evaluation metrics
    r2_current = r2_score(y_val_current.numpy().flatten(), y_pred_val_current)
    mse_current = mean_squared_error(y_val_current.numpy().flatten(), y_pred_val_current)
    sharpe_ratio_current = np.mean(y_pred_val_current) / np.std(y_pred_val_current)
    accuracy_current = accuracy_score((y_val_current.numpy().flatten() > 0).astype(int), (y_pred_val_current > 0).astype(int))
    
    # Append the metrics to the results list
    results_pt.append({
        'Train Size': i,
        'R2 Statistic': r2_current,
        'MSE': mse_current,
        'Sharpe Ratio': sharpe_ratio_current,
        'Accuracy': accuracy_current
    })

# Convert the results list to a DataFrame for easier viewing
results_pt_df = pd.DataFrame(results_pt)

# Display the results
results_pt_df



/Users/ldvyyc/miniconda3/lib/python3.9/site-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([80, 1])) that is different to the input size (torch.Size([80])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/ldvyyc/miniconda3/lib/python3.9/site-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([80, 1])) that is different to the input size (torch.Size([80])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/ldvyyc/miniconda3/lib/python3.9/site-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([80, 1])) that is different to the input size (torch.Size([80])). This will likely lead to incorrect results due to broadcasting. Please ensure they have th

,Train Size,R2 Statistic,MSE,Sharpe Ratio,Accuracy
0,796,-19.284624,0.001006,-84.469498,0.5750
1,876,-4.842166,0.000979,-36.571861,0.5375
2,956,-5.687277,0.000833,-33.210926,0.5875
3,1036,-2.579741,0.000706,-21.737122,0.5875
4,1116,-6.416926,0.000583,-28.988644,0.5875
5,1196,-1.738344,0.001160,-12.082410,0.4250
6,1276,-0.557289,0.001009,-8.922707,0.5125
7,1356,-0.583980,0.001049,-6.902042,0.4875
8,1436,-0.430996,0.000787,-8.335023,0.5000
